Importing Libraries:
==

In [74]:
%matplotlib inline

import time
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils import data
from torchvision.transforms import functional as F
import torchvision

from sklearn.metrics import jaccard_score as jsc

from PIL import Image
from matplotlib.path import Path
import os
import pandas as pd

use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')
    device = "cuda"
    torch.set_default_tensor_type('torch.cuda.FloatTensor') 
else:
    print('GPU is not available!')
    device = "cpu"

GPU is not available!


Loading Data:
==

In [75]:
def transform(x):
    return  torchvision.transforms.functional.to_tensor(x)

class DataSet(torch.utils.data.Dataset):
    def float_maker(self, a):
        if a[0]=='(':
            return a[1:]
        if a[-1]==')':
            return a[:-1]
        return a
    def __init__(self):
        super().__init__()
        self.image_names= os.listdir('./Data_ML/image_chips/')
    def __getitem__(self, index):
        image = plt.imread(os.path.join(os.getcwd(),'Data_ML','image_chips',self.image_names[index]))
        mask = pd.read_csv(os.path.join(os.getcwd(),'Data_ML','target_feature_AOI',self.image_names[index][:-4]+'.csv'))
        coordinates = mask.WKT.values[0][16:-3].split(',')
        # convert coordinates into a masked image (750x750x1)
        points = ([tuple(self.float_maker(i)  for i in x.split()) for x in coordinates])
        h , w = 750, 750
        point_path = Path(points)
        x, y = np.mgrid[:h, :w]
        y = - y
        coors=np.hstack((x.reshape(-1, 1), y.reshape(-1,1)))
        masked_image = (point_path.contains_points(coors))
        first = torchvision.transforms.Resize((768,768))(transform(image))
        second = (torchvision.transforms.functional.to_tensor(masked_image.reshape(h,w)).int())
        return (first,second)
    def __len__(self):
        return 100 #cause we have 100 samples, change this if needed

In [77]:
# input training images are 750x750x3 and are 100 in number
# test image is 5x5 images (hence 3750x3750x3), we need to output a masked image with 0s and 1s
input_channels = 3 
max_channels = 512
output_classes = 1

batch_size = 8 #make it 8 once bug fixing is over
train_size = 100 # divide train:val by 70:30 probably
test_size = 25 # if we break the super test image into individual images to run our model on


train_dataLoader = data.DataLoader(
    DataSet(), #need to get DataSet to make train and CV both
    batch_size = batch_size,
    shuffle = True,
    num_workers= 0
)

Creating Module:
==

In [78]:
class UNetConv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv2 = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(output_channels, output_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv2(x)
class DownConv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv_down = nn.Sequential(
            nn.MaxPool2d(2, 2),
            UNetConv(input_channels, output_channels)
        )
    def forward(self, x):
        return self.conv_down(x)
class UpConv(nn.Module): #whatever is commented is the actual correct code, the line above it is bug fixing ## torch.cat is giving issues
    def __init__(self, input_channels, output_channels, padding=0):
        super().__init__()
        self.conv_up = nn.Sequential(
            nn.ConvTranspose2d(input_channels , input_channels // 2, kernel_size=2, stride=2, padding = padding),
        )
        self.conv_level = nn.Sequential(
            #UNetConv(input_channels // 2,output_channels) 
            UNetConv(input_channels,output_channels) 
        )
    def forward(self, x1, x2):
        #return self.conv_level(self.conv_up(x1))
        return self.conv_level(torch.cat([x2,self.conv_up(x1)],dim = 1)) 
class LastConv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv_final = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size=1)
        )
    def forward(self, x):
        return self.conv_final(x)
class UNet(nn.Module):
    def __init__(self, input_channels, max_channels):
        super(UNet, self).__init__()
        self.current_channels = max_channels // 2**4
        self.start = UNetConv(input_channels, self.current_channels)
        self.current_channels = self.current_channels*2
        self.down1 = DownConv(self.current_channels // 2 , self.current_channels)
        self.current_channels = self.current_channels*2
        self.down2 = DownConv(self.current_channels // 2 , self.current_channels)
        self.current_channels = self.current_channels*2
        self.down3 = DownConv(self.current_channels // 2 , self.current_channels)
        self.current_channels = self.current_channels*2
        self.down4 = DownConv(self.current_channels // 2 , self.current_channels)
        self.current_channels = self.current_channels // 2
        self.up1 = UpConv(self.current_channels * 2, self.current_channels) 
        self.current_channels = self.current_channels // 2
        self.up2 = UpConv(self.current_channels * 2, self.current_channels)
        self.current_channels = self.current_channels // 2
        self.up3 = UpConv(self.current_channels * 2, self.current_channels)
        self.current_channels = self.current_channels // 2
        self.up4 = UpConv(self.current_channels * 2, self.current_channels)
        self.final = LastConv(self.current_channels,output_classes)
    def forward(self, x):
        x1 = self.start(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        #x3 = F.pad(x3, (1,1,0,0))
        x4 = self.down3(x3)
        #x4 = F.pad(x4, (1,0,0,1))
        x5 = self.down4(x4)
        #x5 = F.pad(x5, (0,0,1,1))
        y1 = self.up1(x5,x4)
        y2 = self.up2(y1,x3)
        y3 = self.up3(y2,x2)
        x = self.up4(y3,x1)
        return torch.sigmoid(self.final(x))

Model = UNet(input_channels, max_channels)

Training Model:
==

In [79]:
epochs = 5

class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 1e-5
    def forward(self, y_pred, y_true):
        print(y_pred.shape, y_true.shape)
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (y_pred.sum() + y_true.sum() + self.smooth)
        return 1. - dsc

loss_func = DiceLoss()
optimizer = optim.Adam(Model.parameters(),lr=0.01)
loss_sanity = jsc
trainLoss = []
sanityLosss = []
for epoch in range(epochs):  
    epochStart = time.time()
    runningLoss = 0.0
    sanityLoss = 0.0
    i = 0
    for inputs, labels in tqdm(train_dataLoader): 
        if(i==3):
            break
        #i+=1
        optimizer.zero_grad()  
        outputs = Model(inputs) 
        outputs = torchvision.transforms.Resize((750,750))(outputs)
        loss = loss_func(outputs, labels)
        loss.backward() 
        loss_san = loss_sanity(torch.round(outputs).detach().numpy().reshape(-1), labels.detach().numpy().reshape(-1))
        nn.utils.clip_grad_norm_(Model.parameters(), 50)
        optimizer.step()
        runningLoss += loss.item()
        sanityLoss += loss_san.item()
    runningLoss /= len(inputs) 
    sanityLoss /= len(inputs)
    sanityLosss.append(sanityLoss)
    trainLoss.append(runningLoss)
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f} ; Sanity Loss: {:.6f} ; Time consumed: {:.0f}m {:.0f}s '\
        .format(epoch + 1, epochs, runningLoss, sanityLoss, epochEnd//60, epochEnd%60))   
print('Finished Training')

Testing Model:
==

In [ ]:
test_loss = 0.0
for inputs, labels in tqdm(test_dataLoader):
    outputs = Model(inputs)
    loss = loss_func(outputs, labels)
    test_loss += loss.item()
test_loss /= len(inputs)
print(test_loss)

Error: Session cannot generate requests